In [ ]:
!git clone "https://github.com/kakaobrain/jejueo.git"

In [ ]:
!git clone "https://github.com/pytorch/fairseq"

In [ ]:
!pip install sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

%cd '/content/gdrive/MyDrive/Tobigs/NLP_conference/'
%cd fairseq

In [ ]:
!pip install --editable ./

In [ ]:
cd /content/gdrive/MyDrive/Tobigs/NLP_conference/jejueo/translation

# STEP 1. bpe segment for training

In [ ]:
!python bpe_segment.py --jit jit --vocab_size 4000

# STEP 2. fairseq-prepro

In [ ]:
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/gdrive/MyDrive/Tobigs/NLP_conference/fairseq/"

! echo $PYTHONPATH

In [ ]:
cd /content/gdrive/MyDrive/Tobigs/NLP_conference/jejueo/translation

In [ ]:
!python prepro.py --src ko --tgt je --vocab_size 4000    # 표준어 -> 제주어

# STEP 3. fairseq-train

In [ ]:
!fairseq-train data/4k/ko-je-bin \
    --arch transformer       \
    --optimizer adam \
    --lr 0.0005 \
    --label-smoothing 0.1 \
    --dropout 0.3       \
    --max-tokens 4000 \
    --stop-min-lr '1e-09' \
    --lr-scheduler inverse_sqrt       \
    --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy       \
    --max-epoch 1 \
    --warmup-updates 4000 \
    --warmup-init-lr '1e-07'    \
    --adam-betas '(0.9, 0.98)'       \
    --save-dir train/4k/ko-je-bin/ckpt  \
    --save-interval 1

# STEP 4. fairseq-generate

In [ ]:
!fairseq-generate data/4k/ko-je-bin \
  --path train/4k/ko-je-bin/ckpt/checkpoint_best.pt \
  --source-lang ko --target-lang je \
  --valid-subset 'valid' \
  --gen-subset 'test' \
  --beam 5 \
  --remove-bpe 'sentencepiece'\
  --results-path prediction

In [ ]:
!grep '^H' prediction/generate-test.txt | cut -f3- > prediction/gen.out.sys # 예측된 문장 (H)
!grep '^T' prediction/generate-test.txt | cut -f2- > prediction/gen.out.ref # 타겟(정답) 문장 (T)

# STEP6. Evaluation

In [ ]:
!fairseq-score \
--sys prediction/gen.out.sys \
--ref prediction/gen.out.ref